In [1]:
import pandas as pd
from methods.make_single_df_from_bid_ask import make_single_df_from_bid_ask
# from methods.make_5min_volume_from_1_min_volume import make_5min_volume_from_1min_volume
from machine_learning_models.lstm.eurusd.lstm_predict_candle import predict_candle
# from machine_learning_models.lstm.eurusd.lstm_use_prediction import use_prediction
from machine_learning_models.lstm.lstm_collect_predictions_into_dataframe import collect_predictions_into_dataframe
# import torch

In [2]:
# Display all rows
pd.set_option('display.max_rows', None)

# Display all columns
pd.set_option('display.max_columns', None)

# Prevent column width truncation
pd.set_option('display.max_colwidth', None)

# Display full content without wrapping
pd.set_option('display.expand_frame_repr', False)

In [3]:
BASE_DIR = 'hist_data/'
TIME_SERIES_FOLDER = 'EURUSD/'
BID_FOLDER = '5_min/Bid/'
ASK_FOLDER = '5_min/Ask/'

BASE_DIR_LSTM = 'machine_learning_models/lstm/eurusd/pickle_files'
LSTM_MAX_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_max_values-2025-04-07.pkl'
LSTM_MIN_VAL = BASE_DIR_LSTM + '/lstm_regressor_predict_min_values-2025-04-07.pkl'
SCALER_X_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_min.pkl'
SCALER_X_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_x_max.pkl'
SCALER_Y_MIN = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_min.pkl'
SCALER_Y_MAX = BASE_DIR_LSTM + '/lstm_regressor_scaler_y_max.pkl'

In [4]:
df_joined_train, df_joined_test, df_joined_val = make_single_df_from_bid_ask(
    base_dir=BASE_DIR,
    time_series_folder=TIME_SERIES_FOLDER
)

In [5]:
print(len(df_joined_train), len(df_joined_test), len(df_joined_val))

13845 4615 4616


In [6]:
columns_for_y = [
        "open_plus_5min",
        "high_plus_5min",
        "low_plus_5min",
        "close_plus_5min",
        "open_plus_10min",    
        "high_plus_10min",
        "low_plus_10min",
        "close_plus_10min",
        "open_plus_15min",
        "high_plus_15min",
        "low_plus_15min",
        "close_plus_15min",
        "open_plus_20min",
        "high_plus_20min",
        "low_plus_20min",
        "close_plus_20min",
        "open_plus_25min",
        "high_plus_25min",
        "low_plus_25min",
        "close_plus_25min",
        "open_plus_30min",
        "high_plus_30min",
        "low_plus_30min",
        "close_plus_30min",
        "open_plus_35min",
        "high_plus_35min",
        "low_plus_35min",
        "close_plus_35min",
        "open_plus_40min",
        "high_plus_40min",
        "low_plus_40min",
        "close_plus_40min"
    ]

In [ ]:
for col in columns_for_y:
    predict_candle(df=df_joined_train, test_df=df_joined_test, validation_df=df_joined_val, base_dir=BASE_DIR_LSTM, column_for_y=col)

In [ ]:
dataframe_actual = df_joined_val.tail(8)

In [ ]:
len(df_joined_val)

In [ ]:
dataframe_line = df_joined_val.iloc[4547:4608]

In [ ]:
len(dataframe_line)

In [ ]:
dataframe_predicted = collect_predictions_into_dataframe(dataframe_line=dataframe_line, base_dir_lstm=BASE_DIR_LSTM)

In [ ]:
dataframe_actual[['open', 'high', 'low', 'close']]

In [ ]:
dataframe_predicted

In [ ]:
# predict_min_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MIN, scaler_y_dir=SCALER_Y_MIN, model_dir=LSTM_MIN_VAL)

In [ ]:
# predict_max_stats(df_joined_train, df_joined_test, df_joined_val, scaler_x_dir=SCALER_X_MAX, scaler_y_dir=SCALER_Y_MAX, model_dir=LSTM_MAX_VAL)